<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=07d1692b879e2e9d3f44cb01d987c1bca297437818e9f28b55b3778692ed5c5e
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2


    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-24 09:23:11
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.42 C
-------------------
Today PnL: 25.61 K (0.18%)
Current PnL: -25.00 L (-16.15%)
CY Booked + Current PnL: -10.71 L (-6.92%)
-------------------
Total profit:  1.61 L
Total loss:  -26.62 L
-------------------
Total Booked + Current PnL: 16.27 L (12.73%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.06%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 89.64 L (63.1%)
Deployed:  1.28 C
Current:  1.42 C
CAGR/XIRR %: 5.82%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.25,-18.92,23.46,0.09,19168.0,-19072.0,81705.0,89.54,40.0,M-SC,2.62,253.0,-0.99,0.58,7.34,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,0.38,8.81,10.00,19.69,20864.0,16893.0,208635.0,-1.77,55.0,X-MC,2.48,78.0,0.81,1.48,16.21,XY25,NTT,AC
50,MASFIN,398.61,0.50,-0.61,22.80,22.05,22203.0,-600.0,97380.0,-14.18,58.0,H-SC,6.68,164.0,-0.03,0.69,41.55,XR,ATH,FINANCE
43,ITC,452.00,0.16,-0.00,10.78,10.78,25906.0,-6.0,240312.0,-40.34,59.0,X-LC,2.01,5.0,-0.00,1.70,5.23,X40,NTT,FMCG
17,COALINDIA,484.83,2.31,0.74,18.35,19.23,27813.0,1110.0,151570.0,18.42,78.0,L-LC,2.56,179.0,0.04,1.07,22.17,XY25,ATH,MINING


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SFL,1287.00,4.13,-40.18,111.70,26.63,175203.0,-105367.0,156851.0,18.85,53.0,M-SC,12.25,239.0,-0.60,1.11,7.38,XY24,NTT,MISC
74,TITAGARH,1548.00,3.24,-18.64,79.14,45.74,155926.0,-45151.0,197026.0,8.30,64.0,H-SC,6.36,174.0,-0.29,1.39,28.15,XY24,NTT,ENGINEERING
55,QUESS,424.00,2.56,-26.41,94.11,42.84,45018.0,-17170.0,47836.0,-52.07,57.0,X-SC,33.74,83.0,-0.38,0.34,8.22,XY24,NTT,MISC
17,COALINDIA,484.83,2.31,0.74,18.35,19.23,27813.0,1110.0,151570.0,18.42,78.0,L-LC,2.56,179.0,0.04,1.07,22.17,XY25,ATH,MINING
22,DIXON,18931.72,1.80,-13.01,44.93,26.07,76300.0,-25402.0,169819.0,-57.68,37.0,X-MC,12.38,56.0,-0.33,1.20,5.76,X40N,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,216.0,-2.71,-20.03,53.85,23.03,77118.0,-35863.0,143208.0,55.70,53.0,H-SC,7.67,167.0,-0.47,1.01,42.09,XR,NTT,BANKS
6,ASIANTILES,137.0,-1.55,-4.24,89.72,81.67,80595.0,-3981.0,89829.0,8088.89,67.0,L-SC,12.80,271.0,-0.05,0.64,73.96,XR,NTT,CERAMICS
13,BSOFT,831.7,-1.12,-18.28,84.51,50.79,97137.0,-25712.0,114941.0,4.75,69.0,H-SC,8.43,151.0,-0.26,0.81,33.83,XR,ATH,IT
2,ABBOTINDIA,35195.0,-0.85,-5.88,25.45,18.07,42840.0,-10525.0,168330.0,-19.42,40.0,X-MC,7.27,63.0,-0.25,1.19,11.54,X40,ATH,PHARMA
84,WIPRO,420.0,-0.81,10.41,56.02,72.27,104806.0,17639.0,187087.0,-5.17,69.0,M-LC,3.89,99.0,0.17,1.32,17.23,XR,NTT,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,2.31,0.74,18.35,19.23,27813.0,1110.0,151570.0,18.42,78.0,L-LC,2.56,179.0,0.04,1.07,22.17,XY25,ATH,MINING
50,MASFIN,398.61,0.50,-0.61,22.80,22.05,22203.0,-600.0,97380.0,-14.18,58.0,H-SC,6.68,164.0,-0.03,0.69,41.55,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.55,-2.71,113.70,107.91,163323.0,-4000.0,143644.0,-20.96,52.0,M-SC,11.86,216.0,-0.02,1.02,7.10,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.17,-16.80,20.22,0.02,29365.0,-29330.0,145229.0,121.54,41.0,M-SC,11.40,234.0,-1.00,1.03,26.32,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.25,-18.92,23.46,0.09,19168.0,-19072.0,81705.0,89.54,40.0,M-SC,2.62,253.0,-0.99,0.58,7.34,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,-0.35,-22.03,69.33,32.03,71264.0,-29045.0,102790.0,-15.70,61.0,H-SC,8.94,148.0,-0.41,0.73,15.13,OX40N,NTT,IT
66,SIS,528.00,0.24,-23.94,59.42,21.26,50184.0,-26576.0,84456.0,1992.75,51.0,H-SC,4.80,166.0,-0.53,0.60,14.11,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,2.31,0.74,18.35,19.23,27813.0,1110.0,151570.0,18.42,78.0,L-LC,2.56,179.0,0.04,1.07,22.17,XY25,ATH,MINING
84,WIPRO,420.00,-0.81,10.41,56.02,72.27,104806.0,17639.0,187087.0,-5.17,69.0,M-LC,3.89,99.0,0.17,1.32,17.23,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.81,10.41,56.02,72.27,104806.0,17639.0,187087.0,-5.17,69.0,M-LC,3.89,99.0,0.17,1.32,17.23,XR,NTT,IT
25,FINCABLES,1641.55,-0.55,-2.71,113.70,107.91,163323.0,-4000.0,143644.0,-20.96,52.0,M-SC,11.86,216.0,-0.02,1.02,7.10,OX40N,ATH,CABLES
50,MASFIN,398.61,0.50,-0.61,22.80,22.05,22203.0,-600.0,97380.0,-14.18,58.0,H-SC,6.68,164.0,-0.03,0.69,41.55,XR,ATH,FINANCE
38,INDIAMART,4810.62,-0.04,-2.93,112.96,106.72,135238.0,-3614.0,119722.0,-51.96,44.0,H-SC,4.45,139.0,-0.03,0.85,19.76,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.38,-1.79,38.10,35.63,78416.0,-3757.0,205817.0,-15.85,50.0,H-MC,2.33,119.0,-0.05,1.46,14.58,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.16,-18.02,89.66,55.47,178817.0,-43847.0,199439.0,-27.39,31.0,X-MC,15.60,64.0,-0.25,1.41,2.19,X40N,ATH,FINANCE
8,AWL,485.00,0.15,-24.14,102.34,53.50,234271.0,-72837.0,228914.0,-63.25,33.0,X-MC,5.88,58.0,-0.31,1.62,1.68,XY24,NTT,FMCG
34,HONAUT,58357.33,0.12,-17.98,74.75,43.32,99851.0,-29292.0,133580.0,-29.54,33.0,X-SC,5.96,90.0,-0.29,0.94,2.56,X40N,ATH,ELECTRICAL
3,ACC,3906.00,-0.35,-26.54,123.46,64.15,215808.0,-63151.0,174800.0,-57.19,35.0,X-SC,4.19,82.0,-0.29,1.24,0.00,XY24,BTT,CEMENT
22,DIXON,18931.72,1.80,-13.01,44.93,26.07,76300.0,-25402.0,169819.0,-57.68,37.0,X-MC,12.38,56.0,-0.33,1.20,5.76,X40N,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,0.25,-2.84,24.68,21.14,54515.0,-6452.0,220887.0,-9.65,47.0,X-MC,1.03,74.0,-0.12,1.56,24.50,XY24,NTT,PAINTS
15,CAMS,950.0,0.83,0.43,23.94,24.48,60555.0,1089.0,252945.0,-80.87,51.0,X-SC,1.31,86.0,0.02,1.79,23.67,X40N,NTT,MISC
78,UNITDSPR,1644.0,-0.73,5.14,14.86,20.77,36583.0,12040.0,246184.0,-3.99,51.0,X-MC,1.98,62.0,0.33,1.74,12.42,X40N,NTT,BREWERIES
43,ITC,452.0,0.16,-0.00,10.78,10.78,25906.0,-6.0,240312.0,-40.34,59.0,X-LC,2.01,5.0,-0.00,1.70,5.23,X40,NTT,FMCG
20,DABUR,735.0,0.28,-3.31,48.62,43.70,116618.0,-8211.0,239857.0,-11.25,38.0,X-MC,2.12,72.0,-0.07,1.70,11.24,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.35,-26.54,123.46,64.15,215808.0,-63151.0,174800.0,-57.19,35.0,X-SC,4.19,82.0,-0.29,1.24,0.00,XY24,BTT,CEMENT
19,COLPAL,3726.84,0.17,-19.89,76.65,41.51,161709.0,-52395.0,210970.0,-4.54,41.0,X-MC,7.85,61.0,-0.32,1.49,1.07,XY25,ATH,FMCG
11,BATAINDIA,2096.00,0.15,-38.60,120.21,35.20,94965.0,-49671.0,78999.0,3.54,37.0,X-SC,17.00,92.0,-0.52,0.56,1.33,X40,NTT,FOOTWEAR
8,AWL,485.00,0.15,-24.14,102.34,53.50,234271.0,-72837.0,228914.0,-63.25,33.0,X-MC,5.88,58.0,-0.31,1.62,1.68,XY24,NTT,FMCG
9,BAJAJHFL,181.50,-0.16,-18.02,89.66,55.47,178817.0,-43847.0,199439.0,-27.39,31.0,X-MC,15.60,64.0,-0.25,1.41,2.19,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,2.56,-26.41,94.11,42.84,45018.0,-17170.0,47836.0,-52.07,57.0,X-SC,33.74,83.0,-0.38,0.34,8.22,XY24,NTT,MISC
59,RELAXO,1176.00,0.73,-47.37,189.37,52.31,144684.0,-68757.0,76403.0,-43.96,46.0,X-SC,7.33,91.0,-0.48,0.54,2.27,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.15,-38.60,120.21,35.20,94965.0,-49671.0,78999.0,3.54,37.0,X-SC,17.00,92.0,-0.52,0.56,1.33,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.41,-1.21,25.60,24.08,32105.0,-1541.0,125409.0,-9.18,47.0,X-SC,6.43,89.0,-0.05,0.89,18.16,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.12,-17.98,74.75,43.32,99851.0,-29292.0,133580.0,-29.54,33.0,X-SC,5.96,90.0,-0.29,0.94,2.56,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.24,-9.21,32.94,20.71,113122.0,-34819.0,343418.0,-21.68,68.0,X-LC,3.21,1.0,-0.31,2.43,14.75,X40,ATH,IT
41,INFY,1972.00,-0.73,12.75,19.07,34.24,67273.0,39883.0,352771.0,-12.06,65.0,X-LC,6.13,2.0,0.59,2.50,22.08,X40,NTT,IT
75,TMPV,600.00,-0.19,-25.69,65.49,22.98,109932.0,-58037.0,167861.0,-23.08,53.0,X-LC,6.37,3.0,-0.53,1.19,5.58,XY24,NTT,AUTO
81,VBL,671.64,-0.74,-3.40,40.45,35.67,123410.0,-10750.0,305092.0,-14.83,54.0,X-LC,3.46,4.0,-0.09,2.16,9.98,X40N,ATH,FMCG
43,ITC,452.00,0.16,-0.00,10.78,10.78,25906.0,-6.0,240312.0,-40.34,59.0,X-LC,2.01,5.0,-0.00,1.70,5.23,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.76,-37.25,111.79,32.89,54156.0,-28762.0,48444.0,-698.49,55.0,L-MC,6.81,259.0,-0.53,0.34,33.41,XR,NTT,BANKS
56,RAJESHEXPO,518.00,1.50,-56.27,128.84,0.07,77285.0,-77192.0,59985.0,2098.23,72.0,L-SC,4.11,268.0,-1.00,0.42,49.19,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-1.55,-4.24,89.72,81.67,80595.0,-3981.0,89829.0,8088.89,67.0,L-SC,12.80,271.0,-0.05,0.64,73.96,XR,NTT,CERAMICS
50,MASFIN,398.61,0.50,-0.61,22.80,22.05,22203.0,-600.0,97380.0,-14.18,58.0,H-SC,6.68,164.0,-0.03,0.69,41.55,XR,ATH,FINANCE
13,BSOFT,831.70,-1.12,-18.28,84.51,50.79,97137.0,-25712.0,114941.0,4.75,69.0,H-SC,8.43,151.0,-0.26,0.81,33.83,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.00,1.50,-56.27,128.84,0.07,77285.0,-77192.0,59985.0,2098.23,72.0,L-SC,4.11,268.0,-1.00,0.42,49.19,OX40N,NTT,JEWELLERY
13,BSOFT,831.70,-1.12,-18.28,84.51,50.79,97137.0,-25712.0,114941.0,4.75,69.0,H-SC,8.43,151.0,-0.26,0.81,33.83,XR,ATH,IT
61,ROUTE,2227.21,0.00,-45.73,211.52,69.05,152739.0,-60856.0,72210.0,-56.18,55.0,H-SC,20.59,140.0,-0.40,0.51,11.36,SR,ATH,IT
6,ASIANTILES,137.00,-1.55,-4.24,89.72,81.67,80595.0,-3981.0,89829.0,8088.89,67.0,L-SC,12.80,271.0,-0.05,0.64,73.96,XR,NTT,CERAMICS
17,COALINDIA,484.83,2.31,0.74,18.35,19.23,27813.0,1110.0,151570.0,18.42,78.0,L-LC,2.56,179.0,0.04,1.07,22.17,XY25,ATH,MINING


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.33
1,25,44.78
2,50,76.55


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.00
MC    30.76
LC    24.25
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.84
X40      22.98
X40N     12.95
XR        9.90
XY25      9.13
AR        8.92
OX40N     7.52
SR        1.01
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.21
H-SC    23.85
X-LC    20.75
M-SC    11.59
X-SC     8.01
H-MC     4.82
L-SC     1.55
M-MC     1.39
M-LC     1.32
H-LC     1.11
L-LC     1.07
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.18,-7.46,42.01
IT,13.10,-13.52,70.59
FINANCE,9.56,-17.83,69.08
MISC,7.34,-25.31,76.58
ELECTRICAL,5.95,-11.47,52.30
PAINTS,5.77,-13.05,29.85
INSURANCE,4.75,-1.12,35.73
PHARMA,4.06,-2.42,34.66
AUTO,2.86,-29.89,74.92


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3128594.0,21
AR,1310964.0,10
XR,1258955.0,13
X40,975120.0,14
X40N,854808.0,9
OX40N,729624.0,10
XY25,350653.0,6
SR,277845.0,2
MH,77369.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3604401.0,25
M-SC,1419284.0,15
X-MC,1404563.0,16
X-LC,795252.0,11
X-SC,786931.0,8
H-MC,398623.0,3
L-SC,242402.0,3
M-LC,104806.0,1
H-LC,76971.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1253357.0      6
           AR         901783.0      5
M-SC       XY24       801527.0      6
H-SC       XR         761458.0      7
X-MC       X40        483970.0      7
           XY24       405404.0      3
           X40N       332616.0      4
H-SC       OX40N      332589.0      4
M-SC       OX40N      319750.0      5
X-SC       X40N       305090.0      3
X-LC       X40        302240.0      5
X-SC       XY24       292931.0      3
H-SC       SR         277845.0      2
X-LC       X40N       217102.0      2
H-MC       AR         207621.0      2
           XY24       191002.0      1
X-SC       X40        188910.0      2
X-LC       XY24       184373.0      2
X-MC       XY25       182573.0      2
M-SC       XR         173418.0      2
L-SC       XR         165117.0      2
M-SC       AR         124589.0      2
M-LC       XR         104806.0      1
X-LC       XY25        91537.0      2
H-SC       MH          77369.0      1
L-SC       OX40N       77285.0      1
H-LC       AR          76971.0      1
L-MC       XR          54156.0      1
M-MC       XY25        48730.0      1
L-LC       XY25        27813.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
